In [44]:
import xarray as xr
import numpy as np
import rioxarray
import h5netcdf
import h5py
import netCDF4

import hvplot.xarray
import geoviews as gv

import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import matplotlib as mpl

In [40]:
measures_vx = rioxarray.open_rasterio('data/greenland_vel_mosaic200_2014_2015_vx_v02.1.tif').rename('vx')
measures_vy = rioxarray.open_rasterio('data/greenland_vel_mosaic200_2014_2015_vy_v02.1.tif').rename('vy')
surface_velocity = xr.merge([measures_vx, measures_vy])

age_model = xr.open_dataset('data/RRRAG4_Greenland_1993_2013_01_age_grid.nc',
                               engine='h5netcdf').rename(
                                   {'number of grid points in x-direction': 'x', 'number of grid points in y-direction': 'y', 'number of vertical layers': 'layers', 'number of isochrones': 'original_isochrones',
                                    'x': 'x_km', 'y': 'y_km'}
                                   )
# Convert km to meters for consistency with the rest of the world
age_model = age_model.assign_coords({'x':age_model.x_km[:,0]*1000, 'y':age_model.y_km[0,:]*1000})

bedmachine = xr.open_dataset('data/BedMachineGreenland-v5.nc')

In [109]:
isochrone_ages = np.array([1000, 10000, 20000, 500000])

def interpolate_age_depths(age_norm, age_norm_depths = age_model['depth_norm'].to_numpy(), query_ages = isochrone_ages):
    return np.interp(query_ages, age_norm, age_norm_depths, left=np.nan, right=np.nan)

isochrones_norm = xr.apply_ufunc(interpolate_age_depths, age_model['age_norm'], input_core_dims=[['layers']], output_core_dims=[['isochrone']], vectorize=True)
isochrones_norm = isochrones_norm.assign_coords({'isochrone': isochrone_ages})

In [113]:
layer_elevation = (isochrones_norm * age_model['thick']) + bedmachine['bed'].interp(x=age_model['x'], y=age_model['y'])

In [114]:
layer_elevation.hvplot.quadmesh('x', 'y', crs=crs_3413, rasterize=True, coastline=True, height=500, groupby=['isochrone'])
# TODO : haven't sanity checked this. see if it makes sense. Then work on layer slopes estimates

BokehModel(combine_events=True, render_bundle={'docs_json': {'b41b8c8b-7b30-4a26-881a-35418819072e': {'version…

In [111]:
isochrones_norm.hvplot.quadmesh('x', 'y', crs=crs_3413, rasterize=True, coastline=True, height=500, groupby=['isochrone'])

BokehModel(combine_events=True, render_bundle={'docs_json': {'9b0b6015-b0b7-4019-ae8e-5f311fda2168': {'version…

In [101]:
#isochrones_norm.to_dataset().to(hv.quadmesh, 'x', 'y', groupby=['isochrone'], crs=crs_3413, rasterize=True, coastline=True)
isochrones_norm.assign_coords({'isochrone': [1,2,3,4]}).hvplot.quadmesh('x', 'y', crs=crs_3413, rasterize=True, coastline=True, groupby=['isochrone'])

BokehModel(combine_events=True, render_bundle={'docs_json': {'7fd940de-5d6f-4689-86d0-7f9cc1910124': {'version…

Task exception was never retrieved
future: <Task finished name='Task-58' coro=<Callback.process_on_change() done, defined at /home/thomas/apps/miniconda3/envs/rg2/lib/python3.11/site-packages/holoviews/plotting/bokeh/callbacks.py:328> exception=UnsetValueError("figure(id='p3041', ...).inner_height doesn't have a value set")>
Traceback (most recent call last):
  File "/home/thomas/apps/miniconda3/envs/rg2/lib/python3.11/site-packages/holoviews/plotting/bokeh/callbacks.py", line 347, in process_on_change
    msg[attr] = self.resolve_attr_spec(path, cb_obj)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/thomas/apps/miniconda3/envs/rg2/lib/python3.11/site-packages/holoviews/plotting/bokeh/callbacks.py", line 254, in resolve_attr_spec
    resolved = getattr(resolved, p, None)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/thomas/apps/miniconda3/envs/rg2/lib/python3.11/site-packages/bokeh/core/property/descriptors.py", line 283, in __get__
    raise UnsetValueErr

In [ ]:
# Giving up on matplotlib since I got HV working

# #epsg_3413_maybe = ccrs.NorthPolarStereo(true_scale_latitude=70, central_longitude=70)
# crs_3413 = ccrs.Stereographic(central_latitude=90, central_longitude=-45, true_scale_latitude=70) # I think this is right, but the rest of this still doesn't work
# fig, ax = plt.subplots(subplot_kw={'projection': crs_3413})

# ax.coastlines()
# #ax.set_xlim(-6e6, 9e6)
# #ax.set_ylim(-3e7, -7e6)
# #ax.set_aspect('equal')
# isochrones_norm.loc[{'isochrone': 1}].plot(ax=ax, transform=crs_3413)

In [ ]:
layer_idx = 0
normalized_layer_depth = age_model['age_norm'].loc['number of vertical layers' == layer_idx]
# Note: bedmachine x and y units are EPSG:3413 in meters, age_model x and y units are EPSG:3413 in km
layer_elevation = (normalized_layer_depth * age_model['thick']) #+ bedmachine['bed'].interp(x=age_model['x']*1e3, y=age_model['y']*1e3)


In [ ]:
normalized_layer_depth.plot()

In [ ]:
age_model

In [ ]:
age_model['age_norm']

In [ ]:
netCDF4.Dataset('data/RRRAG4_Greenland_1993_2013_01_age_grid.nc', 'r').variables['age_iso'][:]

In [ ]:
# Open 'data/RRRAG4_Greenland_1993_2013_01_radiostratigraphy.mat' with h5py
f = h5py.File('data/RRRAG4_Greenland_1993_2013_01_radiostratigraphy.mat', 'r')
list(f['gris_strat']['campaign']['name'][5])

In [ ]:
xr.open_dataset(xr.backends.H5NetCDFStore(f['gris_strat']))

In [ ]:
xr.Dataset(f['gris_strat'])